In [ ]:
#pip install --upgrade --force-reinstall numpy
import numpy as np
import pandas as pd
import pickle

import re
from sklearn import linear_model


import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import ensemble 
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import BaggingRegressor


from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
import xgboost as xgb
from sklearn.model_selection import train_test_split


from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold

from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge

from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing

import os
import warnings
warnings.filterwarnings('ignore') 
from pandas_profiling import ProfileReport

In [ ]:
test_df = pd.read_csv("bikerenttest.csv")
train_df = pd.read_csv("bikerenttrain.csv")

In [ ]:
test_df

In [ ]:
train_df

In [ ]:
sns.factorplot(x="hr",y="count",data=train,kind='bar',size=5,aspect=1.5)


In [ ]:
sns.factorplot(x="month",y="count",data=train,kind='bar',size=5,aspect=1.5)

In [ ]:
new_df=train.copy()

new_df['temp_bin']=np.floor(new_df['temp'])//3

sns.factorplot(x="temp_bin",y="count",data=new_df,kind='bar')

In [ ]:
weather_train = pd.get_dummies(train['weather'], prefix='wea', prefix_sep='_')
weather_test = pd.get_dummies(test['weather'], prefix='wea', prefix_sep='_')

season_train = pd.get_dummies(train['season'], prefix='sea', prefix_sep='_')
season_test = pd.get_dummies(test['season'], prefix='sea', prefix_sep='_')

year_train = pd.get_dummies(train['year'], prefix='year', prefix_sep='_')
year_test = pd.get_dummies(test['year'], prefix='year', prefix_sep='_')

wind_categori_train = pd.get_dummies(train['wind_categori'], prefix='wind_cate', prefix_sep='_')
wind_categori_test = pd.get_dummies(test['wind_categori'], prefix='wind_cate', prefix_sep='_')

humi_categori_train = pd.get_dummies(train['humi_categori'], prefix='humi_cate', prefix_sep='_')
humi_categori_test = pd.get_dummies(test['humi_categori'], prefix='humi_cate', prefix_sep='_')

hr_categori_train = pd.get_dummies(train['hr_categori'], prefix='hr_cate', prefix_sep='_')
hr_categori_test = pd.get_dummies(test['hr_categori'], prefix='hr_cate', prefix_sep='_')

In [ ]:
Train_Master = pd.concat([train,
                          weather_train,
                          season_train,
                          year_train,
                          wind_categori_train,
                          humi_categori_train,
                          hr_categori_train
                         ], axis=1)

In [ ]:
Train_Master = Train_Master.drop(['weather','season','year','datetime','hr_categori','humi_categori','wind_categori'], axis = 1)
Test_Master = Test_Master.drop(['weather','season','year','datetime','hr_categori','humi_categori','wind_categori'], axis = 1)

Train, Test = train_test_split(Train_Master, test_size = 0.3)

In [ ]:
X_train = Train.drop(['count'], axis=1)
Y_train = Train['count']
Y_train_log = Train['count'].apply(lambda x:np.log1p(x))

X_test = Test.drop(['count'], axis=1)
Y_test = Test["count"]
Y_test_log = Test["count"].apply(lambda x:np.log1p(x))

Y_test = Y_test.reset_index().drop('index',axis = 1)
Y_train = Y_train.reset_index().drop('index',axis = 1)

In [ ]:
print(Train_Master.columns, Train_Master.shape)
print(Test_Master.columns, Test_Master.shape)
print([i for i in Train_Master.columns if i not in Test_Master.columns]) 

In [ ]:
def random_forest_autotune(X,y):

    param_test = {
     'max_depth':[9,13,17],
     'max_features':['auto','sqrt'],
     'n_estimators': [500,1000,1500]
    }

    gsearch = GridSearchCV(RandomForestRegressor(),refit=True,
                            param_grid = param_test, scoring='neg_mean_squared_error',iid=True, cv=5)
    gsearch.fit(X,y)
    print('____________________________________________')
    print('tune random forest')
    print('____________________________________________')
    print(gsearch.cv_results_)
    print(gsearch.best_params_)
    print(gsearch.best_score_)
    print(rmsle(gsearch.predict(X),y))

    return gsearch.best_estimator_


In [ ]:
random_forest_param = random_forest_autotune(X_train,Y_train_log)

In [ ]:
predict_met = pd.DataFrame(data={method: [np.nan] * X_train.shape[0] for method in [ 'random forest'
                                                                                   ]})
predict_met['random forest'] = random_forest_param.predict(X_train)

predict_met.head()